In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
skt_url='https://lol.gamepedia.com/Special:RunQuery/MatchHistoryGame?MHG%5Bpreload%5D=Team&MHG%5Blimit%5D=130&MHG%5Bteam%5D=SK%20Telecom%20T1&MHG%5Btextonly%5D=Yes&MHG%5Bspl%5D=yes&pfRunQueryFormName=MatchHistoryGame'
damwon_url='https://lol.gamepedia.com/Special:RunQuery/MatchHistoryGame?MHG%5Bpreload%5D=Team&MHG%5Blimit%5D=120&MHG%5Bteam%5D=DAMWON%20Gaming&MHG%5Btextonly%5D=Yes&MHG%5Bspl%5D=yes&pfRunQueryFormName=MatchHistoryGame'
griffin_url='https://lol.gamepedia.com/Special:RunQuery/MatchHistoryGame?MHG%5Bpreload%5D=Team&MHG%5Blimit%5D=100&MHG%5Bteam%5D=Griffin&MHG%5Btextonly%5D=Yes&MHG%5Bspl%5D=yes&pfRunQueryFormName=MatchHistoryGame'

In [3]:
def content_grab(url):
    r=requests.get(url)
    soup=BeautifulSoup(r.content, "html")
    body_html=soup.find("div", {"class":"mw-body"})
    table_html=body_html.find_all("td")
    parse_html=[i.text.strip() for i in table_html]
    array_html=np.asarray(list(parse_html[:(len(parse_html)-22)])).reshape((-1,25))
    column_name= ['Date', 'Tournament', 'P', 'W/L', 'Side', 'Vs', 'Bans', 
                  'BansVs', 'Picks', 'PicksVs', 'Players', 'Len', 'Team_G', 'Team_K', 'Team_T', 'Team_D', 'Team_B', 
                  'VS_G', 'VS_K', 'VS_T', 'VS_D', 'VS_B', 'SB', 'MH', 'VOD']
    raw_df = pd.DataFrame(array_html, columns=column_name)
    df=raw_df.copy()
    df['datetime']=pd.to_datetime(df['Date'])
    df.drop(['Date'], axis=1, inplace=True)
    df=df[df['datetime']>='2019-01-01']
    df.drop(['Vs','SB', 'MH', 'VOD'], axis=1, inplace=True)
    return df

In [6]:
def count_champs(df, col):
    return pd.Series(df[col].str.cat(sep=',').split(',')).value_counts().head(10)

In [7]:
def champ_count_summary(df):
    Bans=count_champs(df,'Bans').reset_index()
    BansVs=count_champs(df,'BansVs').reset_index()
    Picks=count_champs(df,'Picks').reset_index()
    PicksVs=count_champs(df,'PicksVs').reset_index()
    summary_df=pd.concat([Bans, BansVs, Picks, PicksVs],axis=1)
    summary_df.columns=['Bans', 'Count', 'BansVs', 'Count', 'Picks', 'Count', 'PicksVs', 'Count']
    return summary_df

In [4]:
df=content_grab(skt_url)

In [8]:
champ_count_summary(df)

,Bans,Count,BansVs,Count,Picks,Count,PicksVs,Count
0,Yuumi,40,Jayce,38,Lee Sin,32,Tahm Kench,31
1,Irelia,29,Ezreal,33,Braum,30,Aatrox,26
2,Kennen,27,Tahm Kench,29,Ezreal,30,Kai'Sa,22
3,Karma,26,Aatrox,27,Tahm Kench,27,Ezreal,20
4,Akali,25,Lee Sin,26,Aatrox,26,Sylas,19
5,Thresh,22,LeBlanc,25,Sylas,22,Jarvan IV,19
6,Vladimir,21,Kalista,24,Jarvan IV,21,Jayce,18
7,Olaf,20,Sylas,23,Varus,19,Xayah,18
8,Kalista,17,Lucian,21,Galio,18,Galio,17
9,Tahm Kench,17,Sejuani,21,Akali,18,Braum,16


In [17]:
df_win=df[df['W/L']=='Win']

In [18]:
champ_count_summary(df_win)

,Bans,Count,BansVs,Count,Picks,Count,PicksVs,Count
0,Yuumi,29,Jayce,33,Ezreal,24,Tahm Kench,22
1,Kennen,22,Ezreal,21,Lee Sin,22,Kai'Sa,18
2,Karma,20,Tahm Kench,19,Braum,21,Aatrox,17
3,Irelia,19,Lee Sin,18,Tahm Kench,19,Jarvan IV,16
4,Thresh,18,Aatrox,18,Aatrox,19,Sylas,15
5,Akali,16,LeBlanc,17,Jarvan IV,18,Ezreal,13
6,Olaf,16,Sylas,16,Varus,15,Galio,13
7,Kalista,14,Azir,15,Galio,15,Alistar,12
8,Tahm Kench,12,Kalista,15,Sylas,14,Xayah,11
9,Rek'Sai,11,Varus,15,Akali,13,Akali,10


In [9]:
df_loss=df[df['W/L']=='Loss']

In [10]:
champ_count_summary(df_loss)

,Bans,Count,BansVs,Count,Picks,Count,PicksVs,Count
0,Vladimir,11,Ezreal,12,Lee Sin,10,Aatrox,9
1,Yuumi,11,Tahm Kench,10,Braum,9,Tahm Kench,9
2,Sejuani,10,Galio,9,Tahm Kench,8,Jayce,9
3,Irelia,10,Aatrox,9,Sylas,8,Ryze,8
4,Akali,9,Kalista,9,Aatrox,7,Xayah,7
5,Ryze,6,Lee Sin,8,Sejuani,6,Ezreal,7
6,Jayce,6,LeBlanc,8,Rek'Sai,6,Braum,7
7,Sona,6,Sylas,7,Ezreal,6,Lissandra,6
8,Karma,6,Sejuani,7,Jayce,5,Akali,5
9,Kennen,5,Akali,6,Viktor,5,Draven,5


In [11]:
df_loss_blue=df[(df['W/L']=='Loss') & (df['Side']=='Blue')]

In [12]:
champ_count_summary(df_loss_blue)

,Bans,Count,BansVs,Count,Picks,Count,PicksVs,Count
0,Vladimir,7,Kalista,5,Braum,5,Jayce,6
1,Irelia,7,Aatrox,5,Aatrox,4,Tahm Kench,5
2,Yuumi,6,Ryze,4,Tahm Kench,3,Ezreal,4
3,Akali,4,Sylas,4,Varus,3,Galio,3
4,Tahm Kench,3,Tahm Kench,3,Ezreal,3,Jarvan IV,3
5,Elise,3,Lucian,3,Lee Sin,3,Draven,3
6,Kennen,3,LeBlanc,3,Jayce,3,Aatrox,3
7,Rek'Sai,3,Karma,3,Sejuani,3,Braum,3
8,LeBlanc,3,Sejuani,3,Kennen,3,Akali,2
9,Gragas,2,Akali,3,Viktor,3,Fiora,2


In [25]:
df_blue=df[df['Side']=='Blue']

In [26]:
champ_count_summary(df_blue)

,Bans,Count,BansVs,Count,Picks,Count,PicksVs,Count
0,Yuumi,21,Jayce,21,Braum,19,Tahm Kench,17
1,Irelia,21,Sylas,16,Ezreal,18,Kai'Sa,13
2,Kennen,14,Ezreal,16,Aatrox,18,Jarvan IV,12
3,Olaf,13,Lucian,15,Lee Sin,15,Galio,11
4,Vladimir,13,Kalista,14,Jarvan IV,12,Aatrox,10
5,Thresh,11,Varus,13,Tahm Kench,11,Ezreal,10
6,Tahm Kench,11,Aatrox,12,Galio,10,Xayah,10
7,Rek'Sai,11,Akali,11,Azir,10,Jayce,9
8,Ryze,10,Sejuani,11,Rakan,9,Alistar,8
9,Karma,9,Lee Sin,11,Akali,9,Sylas,8


In [30]:
df_red=df[df['Side']=='Red']

In [31]:
champ_count_summary(df_red)

,Bans,Count,BansVs,Count,Picks,Count,PicksVs,Count
0,Yuumi,19,Tahm Kench,18,Lee Sin,17,Aatrox,16
1,Karma,17,Jayce,17,Tahm Kench,16,Tahm Kench,14
2,Akali,16,Ezreal,17,Sylas,14,Ryze,11
3,Kennen,13,Irelia,16,Varus,12,Sylas,11
4,Lucian,13,Aatrox,15,Ezreal,12,Ezreal,10
5,Sejuani,12,LeBlanc,15,Braum,11,Jayce,9
6,Thresh,11,Lee Sin,15,Akali,9,Akali,9
7,Kalista,10,Galio,12,Xayah,9,Kai'Sa,9
8,Irelia,8,Azir,11,Jarvan IV,9,Braum,8
9,Karthus,8,Neeko,10,Kai'Sa,8,Xayah,8


In [13]:
df_loss_red=df[(df['W/L']=='Loss') & (df['Side']=='Red')]

In [14]:
champ_count_summary(df_loss_red)

,Bans,Count,BansVs,Count,Picks,Count,PicksVs,Count
0,Sejuani,9,Ezreal,10,Lee Sin,7,Ryze,6
1,Sona,5,Galio,8,Sylas,6,Aatrox,6
2,Yuumi,5,Tahm Kench,7,Tahm Kench,5,Xayah,5
3,Jarvan IV,5,Lee Sin,6,Rek'Sai,5,Tahm Kench,4
4,Akali,5,Irelia,6,Braum,4,Lissandra,4
5,Rakan,4,LeBlanc,5,Akali,4,Braum,4
6,Sylas,4,Jarvan IV,4,Corki,4,Rakan,3
7,Ryze,4,Yuumi,4,Ryze,4,Jayce,3
8,Lucian,4,Rek'Sai,4,Aatrox,3,Ezreal,3
9,Karma,4,Sejuani,4,Nautilus,3,Camille,3


In [15]:
df_loss_red.shape[0]

23

In [16]:
df_loss_blue.shape[0]

16

In [21]:
df[df['Side']=='Blue'].shape[0]

61

In [22]:
df[df['Side']=='Red'].shape[0]

63

In [23]:
df_loss_blue.shape[0]/df[df['Side']=='Blue'].shape[0]

0.26229508196721313

In [24]:
df_loss_red.shape[0]/df[df['Side']=='Red'].shape[0]

0.36507936507936506